In [7]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [6]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-5")

In [8]:
from langchain_core.tools import tool
    
@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    #query = _safe_sql(query)
    q = query
    if q.startswith("Error:"):
        return q
    try:
        return db.run(q)
    except Exception as e:
        return f"Error: {e}"


In [9]:
SYSTEM = f"""You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [20]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

#llm = init_chat_model("claude-3-5-sonnet-latest", model_provider="anthropic")
llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=llm,
    tools=[execute_sql],
    prompt=SystemMessage(content=SYSTEM),
)

In [22]:
question = "Which table has the largest number of entries?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Which table has the largest number of entries?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_t2N4ZuQZg8vLSoeCaViLTNw0)
 Call ID: call_t2N4ZuQZg8vLSoeCaViLTNw0
  Args:
    query: SELECT group_concat(name, '|') AS tables FROM sqlite_master WHERE type = 'table' AND name NOT LIKE 'sqlite_%';
================================= Tool Message =================================
Name: execute_sql

[('Album|Artist|Customer|Employee|Genre|Invoice|InvoiceLine|MediaType|Playlist|PlaylistTrack|Track',)]
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_rg9KlHcyTgTNUWJ4NVPw6YoW)
 Call ID: call_rg9KlHcyTgTNUWJ4NVPw6YoW
  Args:
    query: SELECT table_name, row_count FROM (
  SELECT 'Album' AS table_name, COUNT(*) AS row_count FROM Album
  UNION ALL
  SELECT 'Artist' AS table_name, COUNT(*) AS row

In [19]:
question = "Which genre on average has the longest tracks?"

print( agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
))

{'messages': [HumanMessage(content='Which genre on average has the longest tracks?', additional_kwargs={}, response_metadata={}, id='6195ebbb-476e-40ba-bef1-3f14fb55b6a9'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 414, 'prompt_tokens': 233, 'total_tokens': 647, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CLv2NObD9MJdGcPuyZL3NdV17SK9U', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--7eef351d-24ed-4af7-b5a2-cf00da3083ac-0', tool_calls=[{'name': 'execute_sql', 'args': {'query': 'SELECT g.Name AS Genre, AVG(t.Milliseconds) AS avg_ms, ROUND(AVG(t.Milliseconds)/60000.0, 2) AS avg_minutes\nFROM Track t\nJOIN Genre

In [ ]:
question = "Which genre on average has the longest tracks?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
question = "What are all the tables?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
question = "Which ones start with 'a'?"

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [13]:
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage

# Create an AI message manually (e.g., for conversation history)
ai_msg = AIMessage("I'd be happy to help you with that question!")

# Add to conversation history
messages = [
    SystemMessage("You are a helpful assistant"),
    HumanMessage("Can you help me?"),
    ai_msg,  # Insert as if it came from the model
    HumanMessage("Great! What's 2+2?")
]

#response = model.invoke(messages)

In [16]:
fagent = create_agent(
    model=llm,
    tools=[execute_sql],
    prompt=SystemMessage("You are a helpful")
)

In [18]:
for step in fagent.stream(
    {"messages": messages},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Great! What's 2+2?
================================== Ai Message ==================================

4
